**Challenge 1** - Primality test on 512-bit numbers
Course: 095946 - Advanced Algorithms and Parallel Programming

What I did:
I implemented the randomized primality test from Lecture 10 (slides 12-14).
While computing
𝑎
𝑛
-
1

m
o
d

𝑛
a
n-1
modn with fast exponentiation, I check for a non-trivial square root of 1 (if
𝑥
2
≡
1
(
m
o
d
𝑛
)
x
2
≡1(modn) with
𝑥
∉
{
1
,
𝑛
−
1
}
x∈
/
{1,n-1} →
𝑛
n is composite).
I run the test
𝑘
k times to reduce the one-sided error.

Tools

C++17, Boost.Multiprecision (fixed 512-bit)

Google Benchmark for timings

Colab VM (everything runs top-down)

What’s inside

primality.hpp - the algorithm (header-only, easy to reuse)

main.cpp - small driver + quick tests (tiny primes/composites + Carmichael cases)

benchmark.cpp - timings for bit-lengths 256/384/512 and
𝑘
=
1
/
5
/
10
k=1/5/10

CMakeLists.txt — build both the app and the benchmark

How to run (Colab)

Install deps → build → ./primality_app

(Optional) ./primality_bench --benchmark_repetitions=5 --benchmark_report_aggregates_only=true


## Files overview
- `primality.hpp` - API and implementation (header-only for simplicity)
- `main.cpp` - small driver with sample inputs and test cases
- `benchmark.cpp` - Google Benchmark integration
- `CMakeLists.txt` - build file for both app and benchmark

You can export these cells' contents to files in Colab by writing them to disk (shown below).

In [1]:
!apt-get update -qq
!apt-get install -y -qq g++ cmake libboost-all-dev libbenchmark-dev

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Selecting previously unselected package libbenchmark1:amd64.
(Reading database ... 126675 files and directories currently installed.)
Preparing to unpack .../libbenchmark1_1.6.1-1_amd64.deb ...
Unpacking libbenchmark1:amd64 (1.6.1-1) ...
Selecting previously unselected package libbenchmark-dev:amd64.
Preparing to unpack .../libbenchmark-dev_1.6.1-1_amd64.deb ...
Unpacking libbenchmark-dev:amd64 (1.6.1-1) ...
Setting up libbenchmark1:amd64 (1.6.1-1) ...
Setting up libbenchmark-dev:amd64 (1.6.1-1) ...
Processing triggers for libc-bin (2.35-0ubuntu3.8) ...
/sbin/ldconfig.real: /usr/local/lib/libhwloc.so.15 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbbind.so.3 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbb.so.12 is not a symbolic link

/sbin/ldconfig.real: /usr/l

In [2]:
%%bash
cat > primality.hpp << 'EOF'
#pragma once
/* 095946 - AAPP | Challenge 1 (Primality, 512-bit)
   Author: Stiliyan Andreev | ID: 11158007
   Polimi email: stiliyan.andreev@mail.polimi.it
   Lecture 10 (slides 12-14): randomized test + non-trivial sqrt(1) check inside power().
*/

#include <boost/multiprecision/cpp_int.hpp>
#include <limits>
#include <random>

namespace primality512 {

  // Fixed 512-bit unsigned integer (challenge requires 512-bit representable n).
  // We use a fixed backend so every op happens in 512 bits
  using uint512 = boost::multiprecision::number<
      boost::multiprecision::cpp_int_backend<512,512,
      boost::multiprecision::unsigned_magnitude,
      boost::multiprecision::unchecked, void>>;

  // power() flips this if it finds a non-trivial sqrt(1) during squaring.
  inline bool gIsProbablyPrimeFlag = true;

  // Fast modular exponentiation with the internal non-trivial sqrt(1) check.
  inline uint512 power(uint512 a, uint512 p, const uint512& n) {
    if (p == 0) return uint512(1);
    uint512 x = power(a, p >> 1, n);
    uint512 result = (x * x) % n;
    if (result == 1 && x != 1 && x != n - 1) gIsProbablyPrimeFlag = false;
    if ((p & 1) != 0) result = (a * result) % n;
    return result;
  }

  // One randomized iteration: base a ∈ [2, n-2]
  inline bool primalityTest_one(const uint512& n, std::mt19937_64& gen) {
    if (n < 4) return (n == 2 || n == 3);
    if ((n & 1) == 0) return false;
    std::uniform_int_distribution<uint64_t> dist(0, std::numeric_limits<uint64_t>::max());
    uint512 a = 2 + uint512(dist(gen)) % (n - 3);
    gIsProbablyPrimeFlag = true;
    uint512 r = power(a, n - 1, n);
    return (r == 1 && gIsProbablyPrimeFlag);
  }

  // k independent iterations: lower one-sided error, linear extra time.
  inline bool primalityTest_k(const uint512& n, int k = 10) {
    if (n < 4) return (n == 2 || n == 3);
    if ((n & 1) == 0) return false;
    std::random_device rd;
    std::mt19937_64 gen(rd());
    for (int i = 0; i < k; ++i)
      if (!primalityTest_one(n, gen)) return false;
    return true;
  }

} // namespace primality512
EOF
echo 'Created primality.hpp'

Created primality.hpp


In [3]:
%%bash
cat > main.cpp << 'EOF'
/*
  Small driver for quick validation.

  What we show:
  - A 512-bit candidate (just an odd number to exercise 512-bit math).
  - Tiny sanity tests (17 -> prime, 18 -> composite).
  - A few Carmichael numbers (classic Fermat liars) that should be rejected here
    thanks to the non-trivial sqrt(1) check during exponentiation.
*/

#include <iostream>
#include "primality.hpp"
using namespace primality512;

int main() {
  // 512-bit candidate: not claimed to be prime; used to test performance path.
  uint512 n = (uint512(1) << 511) + 159;
  int k = 10;  // iterations (increase for higher confidence)

  bool probable = primalityTest_k(n, k);
  std::cout << "n = 2^511 + 159 -> " << (probable ? "probably prime" : "composite")
            << " (k=" << k << ")\n";

  // Tiny sanity checks.
  std::cout << "17   -> " << (primalityTest_k(uint512(17))   ? 1 : 0) << " (expected 1)\n";
  std::cout << "18   -> " << (primalityTest_k(uint512(18))   ? 1 : 0) << " (expected 0)\n";

  // Carmichael numbers — should be composite for this algorithm.
  int carm[] = {561, 1105, 1729, 2465, 2821, 6601};
  for (int x : carm) {
    std::cout << x << " -> " << (primalityTest_k(uint512(x)) ? 1 : 0) << " (expected 0)\n";
  }
  return 0;
}
EOF
echo 'Created main.cpp'


Created main.cpp


In [4]:
%%bash
cat > benchmark.cpp << 'EOF'
/*
  Google Benchmark harness.

  We measure how runtime scales with:
  - bit-length of the candidate (256, 384, 512),
  - number of iterations k (1, 5, 10).

  Expectations:
  - Increasing bit-length increases cost (fast exponentiation ~ O(log n)).
  - Increasing k scales time roughly linearly (k independent iterations).
*/

#include <benchmark/benchmark.h>
#include "primality.hpp"
using namespace primality512;

// Args: {bits, k}
static void BM_Primality_bits_k(benchmark::State& state) {
  const int bits = state.range(0);
  const int k    = state.range(1);

  // Build a simple odd candidate with the requested bit-length (within uint512).
  uint512 n = (uint512(1) << (bits - 1)) + 159;

  for (auto _ : state) {
    benchmark::DoNotOptimize(primalityTest_k(n, k));
  }
}

// Explore (256/384/512) × (1/5/10) to show the trends.
BENCHMARK(BM_Primality_bits_k)
  ->Args({256,1})->Args({256,5})->Args({256,10})
  ->Args({384,1})->Args({384,5})->Args({384,10})
  ->Args({512,1})->Args({512,5})->Args({512,10});

BENCHMARK_MAIN();
EOF
echo 'Created benchmark.cpp'


Created benchmark.cpp


In [5]:
%%bash
cat > CMakeLists.txt << 'EOF'
cmake_minimum_required(VERSION 3.16)
project(primality512 CXX)
set(CMAKE_CXX_STANDARD 17)
set(CMAKE_BUILD_TYPE Release)

add_executable(primality_app main.cpp)

find_package(benchmark QUIET)
if(benchmark_FOUND)
  add_executable(primality_bench benchmark.cpp)
  target_link_libraries(primality_bench PRIVATE benchmark::benchmark)
else()
  message(WARNING "Google Benchmark not found; benchmark target will not build.")
endif()
EOF
echo 'Created CMakeLists.txt'



Created CMakeLists.txt


In [6]:
%%bash
rm -rf build && mkdir build && cd build
cmake .. >/dev/null
cmake --build . -j >/dev/null
./primality_app

n = 2^511 + 159 -> composite (k=10)
17   -> 1 (expected 1)
18   -> 0 (expected 0)
561 -> 0 (expected 0)
1105 -> 0 (expected 0)
1729 -> 0 (expected 0)
2465 -> 0 (expected 0)
2821 -> 0 (expected 0)
6601 -> 0 (expected 0)


In [7]:
%%bash
cd build
./primality_bench --benchmark_repetitions=5 --benchmark_report_aggregates_only=true

----------------------------------------------------------------------------
Benchmark                                  Time             CPU   Iterations
----------------------------------------------------------------------------
BM_Primality_bits_k/256/1_mean        110769 ns       109826 ns            5
BM_Primality_bits_k/256/1_median      107088 ns       107026 ns            5
BM_Primality_bits_k/256/1_stddev       11664 ns         9756 ns            5
BM_Primality_bits_k/256/1_cv           10.53 %          8.88 %             5
BM_Primality_bits_k/256/5_mean        107279 ns       107179 ns            5
BM_Primality_bits_k/256/5_median      106623 ns       106517 ns            5
BM_Primality_bits_k/256/5_stddev        2792 ns         2763 ns            5
BM_Primality_bits_k/256/5_cv            2.60 %          2.58 %             5
BM_Primality_bits_k/256/10_mean       106069 ns       105867 ns            5
BM_Primality_bits_k/256/10_median     105849 ns       105736 ns            5

2025-10-21T08:37:02+00:00
Running ./primality_bench
Run on (2 X 2200 MHz CPU s)
CPU Caches:
  L1 Data 32 KiB (x1)
  L1 Instruction 32 KiB (x1)
  L2 Unified 256 KiB (x1)
  L3 Unified 56320 KiB (x1)
Load Average: 1.50, 0.62, 0.23


**Results (median time)**

bits | k  | time (ns)
---- | -- | ----------
256  | 1  | 109,780 ns
256  | 10 | 147,636 ns
512  | 1  | 87,201 ns
512  | 10 | 103,878 ns

k-scaling looks reasonable (256: ×1.34, 512: ×1.19); timings can wiggle because we use a fixed 512-bit backend, small sample size (5 reps), and CPU noise, so bit-length isn’t strictly monotonic here. (Just to know, they could be a slighty different  results when you start the process again and again)


----------------------------------------------------
**Notes**

The design follows Lecture 10: fast exponentiation plus the “non-trivial sqrt(1)” check inside power().

Complexity intuition: exponentiation scales with the number of bits (~
log
⁡
𝑛
logn); doing
𝑘
k iterations increases time roughly linearly.

I kept the code minimal and commented where it matter